In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
train_candidates = pd.DataFrame()
test_candidates = pd.DataFrame()
vall_candidates = pd.DataFrame()

In [3]:
train_chs = pd.read_csv('prep_data/train_choices.csv')
test_chs = pd.read_csv('prep_data/test_choices.csv')
vall_chs = pd.read_csv('prep_data/vall_choices.csv')

In [4]:
train_chs

choice_1_concept               choice_2_concept  \
0                  pretty awful                      very good   
1     practising cooking skills                    put wife in   
2                       at home                  in classrooms   
3                 playing piano                 for performing   
4                  broken watch                   broken watch   
...                         ...                            ...   
7302                     tomato                         cheese   
7303                 low prices           greater availability   
7304                 bland diet  storing food for long periods   
7305                   in weeks                      in months   
7306          driving somewhere          know how to drive car   

           choice_3_concept  
0                its better  
1               put soup in  
2                 on street  
3         singing something  
4          old pocket watch  
...                     ...  
7302              fish fish  
7303         good nutrition  
7304       nice fish dinner  
7305               two year  
7306  required to drive car  

[7307 rows x 3 columns]

In [5]:
train_chs['choices'] = train_chs.apply(lambda x: list(x.values),axis=1)
test_chs['choices'] = test_chs.apply(lambda x: list(x.values),axis=1)
vall_chs['choices'] = vall_chs.apply(lambda x: list(x.values),axis=1)

In [6]:
train_chs

choice_1_concept               choice_2_concept  \
0                  pretty awful                      very good   
1     practising cooking skills                    put wife in   
2                       at home                  in classrooms   
3                 playing piano                 for performing   
4                  broken watch                   broken watch   
...                         ...                            ...   
7302                     tomato                         cheese   
7303                 low prices           greater availability   
7304                 bland diet  storing food for long periods   
7305                   in weeks                      in months   
7306          driving somewhere          know how to drive car   

           choice_3_concept                                            choices  
0                its better              [pretty awful, very good, its better]  
1               put soup in  [practising cooking skills, put wife in, put s...  
2                 on street                [at home, in classrooms, on street]  
3         singing something  [playing piano, for performing, singing someth...  
4          old pocket watch     [broken watch, broken watch, old pocket watch]  
...                     ...                                                ...  
7302              fish fish                        [tomato, cheese, fish fish]  
7303         good nutrition  [low prices, greater availability, good nutrit...  
7304       nice fish dinner  [bland diet, storing food for long periods, ni...  
7305               two year                    [in weeks, in months, two year]  
7306  required to drive car  [driving somewhere, know how to drive car, req...  

[7307 rows x 4 columns]

In [7]:
train_lemma = pd.read_csv('prep_data/train_rank_by_ids.csv')
test_lemma = pd.read_csv('prep_data/test_rank_by_ids.csv')
vall_lemma = pd.read_csv('prep_data/vall_rank_by_ids.csv')

In [8]:
train_lemma['candidates_rank'] = train_lemma['candidates_rank'].apply(ast.literal_eval)
test_lemma['candidates_rank'] = test_lemma['candidates_rank'].apply(ast.literal_eval)
vall_lemma['candidates_rank'] = vall_lemma['candidates_rank'].apply(ast.literal_eval)

In [9]:
train_bge = pd.read_csv('prep_data/train_dream_bge.csv')
test_bge = pd.read_csv('prep_data/test_dream_bge.csv')
vall_bge = pd.read_csv('prep_data/val_dream_bge.csv')

In [10]:
train_bge['candidates'] = train_bge['candidates'].apply(ast.literal_eval)
test_bge['candidates'] = test_bge['candidates'].apply(ast.literal_eval)
vall_bge['candidates'] = vall_bge['candidates'].apply(ast.literal_eval)

In [11]:
train_candidates['candidates'] = train_lemma['candidates_rank'] + train_bge['candidates']
test_candidates['candidates'] =  test_lemma['candidates_rank'] + test_bge['candidates']
vall_candidates['candidates'] =  vall_lemma['candidates_rank'] + vall_bge['candidates'] 

In [12]:
!jupyter server list

Currently running servers:
http://npii00imaa:8888/?token=d119155750b4b59570c1402091d6aefc :: /notebooks


In [13]:
import pickle
candidates = {
    "train_candidates": train_candidates['candidates'].tolist(),
    "test_candidates": test_candidates['candidates'].tolist(),
    "vall_candidates": vall_candidates['candidates'].tolist(),
}

with open('dream_all_candidates.pkl','wb') as f:
    pickle.dump(candidates,f)

In [14]:
def parse_dialog(sentences):
    return '; '.join(sentences)

def parse_request(x):
    return f"{x['dialog']} {x['question']} {x['choice_0']} {x['choice_1']}  {x['choice_2']}"



def make_df(path, is_test=False):
    data = pd.read_json(path,lines=True)
    df = []
    for i in range(len(data)):
        row = data.iloc[i]
        dialog = parse_dialog(row[0])
        for ques in row[1]:
            df.append({
                'dialog_id': i,
                'dialog': dialog,
                'question': ques['question'],
                'choice_0': ques['choice'][0],
                'choice_1': ques['choice'][1],
                'choice_2': ques['choice'][2],
                'labels': -100 if is_test else ques['choice'].index(ques['answer'])
            })
    return pd.DataFrame(df)

train_df = make_df('data/train.statement.jsonl')
test_df = pd.read_csv('dream_test_answers.csv')
vall_df = make_df('data/dev.statement.jsonl')

In [17]:
vall_df

dialog_id                                             dialog  \
0            0  M: I believe that choosing this man will be a ...   
1            1  M: Oh, no, it's Monday again. I always feel ti...   
2            1  M: Oh, no, it's Monday again. I always feel ti...   
3            1  M: Oh, no, it's Monday again. I always feel ti...   
4            2  M: Would you like something to drink, Mum?; W:...   
..         ...                                                ...   
808        515  Justin: Uh, hi. Do you need some help with tho...   
809        515  Justin: Uh, hi. Do you need some help with tho...   
810        515  Justin: Uh, hi. Do you need some help with tho...   
811        515  Justin: Uh, hi. Do you need some help with tho...   
812        515  Justin: Uh, hi. Do you need some help with tho...   

                                              question  \
0                        What is the woman's attitude?   
1                              What day was yesterday?   
2    How many people were invited to the party at L...   
3              With whom did the woman watch the film?   
4              What has the woman ordered for herself?   
..                                                 ...   
808                          What is the woman's name?   
809  The woman spent most of her life in __________...   
810               What does the woman do for a living?   
811            How often does the woman go to the gym?   
812  Which statement is NOT true about the conversa...   

                                     choice_0  \
0                    She agrees with the man.   
1                                     Monday.   
2                                  Seventeen.   
3                                Her brother.   
4                                       Milk.   
..                                        ...   
808                     Her name is Ruth Ann.   
809                                California   
810             She is a computer programmar.   
811        She goes there three times a week.   
812  The man is from a city called Salt Lake.   

                                      choice_1  \
0              She doesn't agree with the man.   
1                                      Sunday.   
2                                     Sixteen.   
3                             Her best friend.   
4                                       Juice.   
..                                         ...   
808                                It's Ronda.   
809                                       Utah   
810                     She works at a school.   
811           She works out four times a week.   
812  The woman gives the man her phone number.   

                                 choice_2  labels  
0            She doesn't know what to do.       1  
1                                Tuesday.       1  
2                                 Twenty.       1  
3                          Her boyfriend.       0  
4                                 Coffee.       0  
..                                    ...     ...  
808           The woman's name is Rachel.       2  
809                              New York       1  
810              She has a job at a bank.       1  
811      She exercises five times a week.       2  
812  The woman knows a lot about history.       1  

[813 rows x 7 columns]

In [18]:
labels = []
for i in range(len(test_df)):
    row = test_df.iloc[i]
    if row['answers'] not in [row['choice_0'], row['choice_1'], row['choice_2']]:
        labels.append(-1)
    else:
        labels.append([row['choice_0'], row['choice_1'], row['choice_2']].index(row['answers']))

In [19]:
test_df['labels'] = labels

In [20]:
train_df['choices_rq'] = train_chs['choices']
test_df['choices_rq'] = test_chs['choices']
vall_df['choices_rq'] = vall_chs['choices']

In [21]:
train_df

dialog_id                                             dialog  \
0             0  W: Well, I'm afraid my cooking isn't to your t...   
1             0  W: Well, I'm afraid my cooking isn't to your t...   
2             1  M: Look at the girl on the bike!; F: Oh, yes s...   
3             2  M: Can you accompany me while I am singing.; W...   
4             3  W: I can't figure out what's wrong with my wat...   
...         ...                                                ...   
7302       4637  W: I've been reading about a new snack food. I...   
7303       4637  W: I've been reading about a new snack food. I...   
7304       4637  W: I've been reading about a new snack food. I...   
7305       4637  W: I've been reading about a new snack food. I...   
7306       4638  W: Do people drive on the right in England?; M...   

                                               question  \
0       What does the man think of the woman's cooking?   
1                What does the woman ask the man to do?   
2                            Where are the two persons?   
3               What does the man want the woman to do?   
4                               What does the man mean?   
...                                                 ...   
7302  Which of the following is NOT mentioned in the...   
7303  What would be a good reason for the man to buy...   
7304  According to the woman, what is an advantage o...   
7305            When will the new product be available?   
7306                       What are they talking about?   

                                               choice_0  \
0                                 It's really terrible.   
1                               Improve cooking skills.   
2                                               At home   
3                                    To play the piano.   
4     It will take about one month to repair the watch.   
...                                                 ...   
7302                                            Tomato.   
7303                            Its low purchase price.   
7304                  Its bland flavoring is healthful.   
7305                                    In a few weeks.   
7306                                Driving in England.   

                                     choice_1  \
0                      It's very good indeed.   
1                   Bring his wife next time.   
2                          In their classroom   
3                      To give a performance.   
4     The watch is no longer worth repairing.   
...                                       ...   
7302                                  Cheese.   
7303                   Its wide availability.   
7304            It can be stored a long time.   
7305                     In about two months.   
7306                      How to drive a car.   

                                               choice_2  labels  \
0                        It's better than what he does.       1   
1                                       Have some soup.       1   
2                                         On the street       2   
3                                       To sing a song.       0   
4              It is a good idea to keep the old watch.       1   
...                                                 ...     ...   
7302                                              Fish.       2   
7303                        Its good nutritional value.       2   
7304                     It goes well with fish dishes.       1   
7305                                In about two years.       2   
7306  Whether to have the right to drive a car in En...       0   

                                             choices_rq  
0                 [pretty awful, very good, its better]  
1     [practising cooking skills, put wife in, put s...  
2                   [at home, in classrooms, on street]  
3     [playing piano, for performing, singing someth...  
4        [broken watch, broken watch, old pocket watch]  
...  

In [22]:
train_df['right_concept'] = train_df.apply(lambda x: x['choices_rq'][x['labels']],axis=1)
test_df['right_concept'] = test_df.apply(lambda x: x['choices_rq'][x['labels']],axis=1)
vall_df['right_concept'] = vall_df.apply(lambda x: x['choices_rq'][x['labels']],axis=1)

In [23]:
train_df

dialog_id                                             dialog  \
0             0  W: Well, I'm afraid my cooking isn't to your t...   
1             0  W: Well, I'm afraid my cooking isn't to your t...   
2             1  M: Look at the girl on the bike!; F: Oh, yes s...   
3             2  M: Can you accompany me while I am singing.; W...   
4             3  W: I can't figure out what's wrong with my wat...   
...         ...                                                ...   
7302       4637  W: I've been reading about a new snack food. I...   
7303       4637  W: I've been reading about a new snack food. I...   
7304       4637  W: I've been reading about a new snack food. I...   
7305       4637  W: I've been reading about a new snack food. I...   
7306       4638  W: Do people drive on the right in England?; M...   

                                               question  \
0       What does the man think of the woman's cooking?   
1                What does the woman ask the man to do?   
2                            Where are the two persons?   
3               What does the man want the woman to do?   
4                               What does the man mean?   
...                                                 ...   
7302  Which of the following is NOT mentioned in the...   
7303  What would be a good reason for the man to buy...   
7304  According to the woman, what is an advantage o...   
7305            When will the new product be available?   
7306                       What are they talking about?   

                                               choice_0  \
0                                 It's really terrible.   
1                               Improve cooking skills.   
2                                               At home   
3                                    To play the piano.   
4     It will take about one month to repair the watch.   
...                                                 ...   
7302                                            Tomato.   
7303                            Its low purchase price.   
7304                  Its bland flavoring is healthful.   
7305                                    In a few weeks.   
7306                                Driving in England.   

                                     choice_1  \
0                      It's very good indeed.   
1                   Bring his wife next time.   
2                          In their classroom   
3                      To give a performance.   
4     The watch is no longer worth repairing.   
...                                       ...   
7302                                  Cheese.   
7303                   Its wide availability.   
7304            It can be stored a long time.   
7305                     In about two months.   
7306                      How to drive a car.   

                                               choice_2  labels  \
0                        It's better than what he does.       1   
1                                       Have some soup.       1   
2                                         On the street       2   
3                                       To sing a song.       0   
4              It is a good idea to keep the old watch.       1   
...                                                 ...     ...   
7302                                              Fish.       2   
7303                        Its good nutritional value.       2   
7304                     It goes well with fish dishes.       1   
7305                                In about two years.       2   
7306  Whether to have the right to drive a car in En...       0   

                                             choices_rq  \
0                 [pretty awful, very good, its better]   
1     [practising cooking skills, put wife in, put s...   
2                   [at home, in classrooms, on street]   
3     [playing piano, for performing, singing someth...   
4        [broken watch, broken watch, old pocket watch]   

In [24]:
train_df['candidates'] = train_candidates['candidates']
test_df['candidates'] = test_candidates['candidates']
vall_df['candidates'] = vall_candidates['candidates']

In [25]:
train_df

dialog_id                                             dialog  \
0             0  W: Well, I'm afraid my cooking isn't to your t...   
1             0  W: Well, I'm afraid my cooking isn't to your t...   
2             1  M: Look at the girl on the bike!; F: Oh, yes s...   
3             2  M: Can you accompany me while I am singing.; W...   
4             3  W: I can't figure out what's wrong with my wat...   
...         ...                                                ...   
7302       4637  W: I've been reading about a new snack food. I...   
7303       4637  W: I've been reading about a new snack food. I...   
7304       4637  W: I've been reading about a new snack food. I...   
7305       4637  W: I've been reading about a new snack food. I...   
7306       4638  W: Do people drive on the right in England?; M...   

                                               question  \
0       What does the man think of the woman's cooking?   
1                What does the woman ask the man to do?   
2                            Where are the two persons?   
3               What does the man want the woman to do?   
4                               What does the man mean?   
...                                                 ...   
7302  Which of the following is NOT mentioned in the...   
7303  What would be a good reason for the man to buy...   
7304  According to the woman, what is an advantage o...   
7305            When will the new product be available?   
7306                       What are they talking about?   

                                               choice_0  \
0                                 It's really terrible.   
1                               Improve cooking skills.   
2                                               At home   
3                                    To play the piano.   
4     It will take about one month to repair the watch.   
...                                                 ...   
7302                                            Tomato.   
7303                            Its low purchase price.   
7304                  Its bland flavoring is healthful.   
7305                                    In a few weeks.   
7306                                Driving in England.   

                                     choice_1  \
0                      It's very good indeed.   
1                   Bring his wife next time.   
2                          In their classroom   
3                      To give a performance.   
4     The watch is no longer worth repairing.   
...                                       ...   
7302                                  Cheese.   
7303                   Its wide availability.   
7304            It can be stored a long time.   
7305                     In about two months.   
7306                      How to drive a car.   

                                               choice_2  labels  \
0                        It's better than what he does.       1   
1                                       Have some soup.       1   
2                                         On the street       2   
3                                       To sing a song.       0   
4              It is a good idea to keep the old watch.       1   
...                                                 ...     ...   
7302                                              Fish.       2   
7303                        Its good nutritional value.       2   
7304                     It goes well with fish dishes.       1   
7305                                In about two years.       2   
7306  Whether to have the right to drive a car in En...       0   

                                             choices_rq  \
0                 [pretty awful, very good, its better]   
1     [practising cooking skills, put wife in, put s...   
2                   [at home, in classrooms, on street]   
3     [playing piano, for performing, singing someth...   
4        [broken watch, broken watch, old pocket watch]   

In [26]:
X_dream = train_df['candidates'].tolist() + test_df['candidates'].tolist() + vall_df['candidates'].tolist()
y_dream = train_df['right_concept'].tolist() + test_df['right_concept'].tolist() + vall_df['right_concept'].tolist()

In [27]:
import pickle
graph_state = pickle.load(open('graph.pkl','rb'))
state = pickle.load(open('csqa_data/all_candsV3.pkl','rb'))

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f2bca728f10>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [28]:
def make_csqa_df(path,is_test=False):
    data = pd.read_json(path,lines=True)
    df = pd.DataFrame()
    
    df['question_state'] = data['question'].apply(lambda x: x['question_concept'])
    df['id'] = data['id']
    df['question'] = data['question'].apply(lambda x: x['stem'])
    df['choice_A'] = data['question'].apply(lambda x: x['choices'][0]['text'])
    df['choice_B'] = data['question'].apply(lambda x: x['choices'][1]['text'])
    df['choice_C'] = data['question'].apply(lambda x: x['choices'][2]['text'])
    df['choice_D'] = data['question'].apply(lambda x: x['choices'][3]['text'])
    df['choice_E'] = data['question'].apply(lambda x: x['choices'][4]['text'])
    
    if not is_test:
        df['label'] = data['answerKey'].map({'A':0,'B':1,'C':2,'D':3,'E':4})
    else:
        df['label'] = -100
    return df

In [29]:
train_data = make_csqa_df('csqa_data/train.statement.jsonl')
vall_data = make_csqa_df('csqa_data/dev.statement.jsonl')
test_data = make_csqa_df('csqa_data/test.statement.jsonl',is_test=True)

In [30]:
train_data

question_state                                id  \
0         punishing  075e483d21c29a511267ef62bedc0461   
1            people  61fe6e879ff18686d7552425a36344c8   
2            choker  4c1cb0e95b99f72d55c068ba0255c54d   
3           highway  02e821a3e53cb320790950aab4489e85   
4             cable  e8a8b3a2061aa0e6d7c6b522e9612824   
...             ...                               ...   
7888            cat  1fe90a4aee405e1aa2279442d28803ae   
7889        landing  01794dde3ca2991615f1aa2f63fb22e3   
7890          grape  21ce6f7c5c3d1ad8cf234988c1ad471f   
7891          climb  6c84e79d0595efd99596faa07c4961d0   
7892           hose  88f1fe6cfbcb1a25f25454341c789463   

                                               question            choice_A  \
0     The sanctions against the school were a punish...              ignore   
1     Sammy wanted to go to where the people were. W...          race track   
2     To locate a choker not located in a jewelry bo...       jewelry store   
3     Google Maps and other highway and street GPS s...       united states   
4     What home entertainment equipment requires cable?         radio shack   
...                                                 ...                 ...   
7888                     What are cats often known for?            whiskers   
7889  As he looked out the window, he knew the landi...  apartment building   
7890  If you really wanted a grape, where would you ...              winery   
7891  What would you do to a rock when climb up a cl...                grab   
7892  His compressor needed a new hose, where did he...         garden shed   

             choice_B       choice_C     choice_D     choice_E  label  
0             enforce  authoritarian      yell at        avoid      0  
1     populated areas     the desert    apartment    roadblock      1  
2                neck    jewlery box  jewelry box     boutique      0  
3              mexico    countryside        atlas       oceans      3  
4          substation        cabinet   television         desk      3  
...               ...            ...          ...          ...    ...  
7888      sharp teeth           purr    four legs  sharp claws      4  
7889        disembark      stairwell      deplane      airport      4  
7890      fruit stand          field      kitchen         food      1  
7891        look down          throw      falling     may fall      0  
7892   hardware store        brothel       garage   greenhouse      1  

[7893 rows x 9 columns]

In [31]:
test_answers = pd.read_csv('answers.csv')
test_df = pd.read_json('csqa_data/test.statement.jsonl',lines=True)
test_data['id'] = test_df['id']
test_data = test_data.merge(test_answers)

In [32]:
test_data['answer'] = test_data.apply(lambda x: [x[f'choice_{i}'] for i in 'ABCDE']['ABCDE'.index(x['answer'])],axis=1)

In [33]:
train_data['answer'] = train_data.apply(lambda x: [x[f'choice_{i}'] for i in 'ABCDE'][x['label']],axis=1)
vall_data['answer'] = vall_data.apply(lambda x: [x[f'choice_{i}'] for i in 'ABCDE'][x['label']],axis=1)

In [34]:
train_data

question_state                                id  \
0         punishing  075e483d21c29a511267ef62bedc0461   
1            people  61fe6e879ff18686d7552425a36344c8   
2            choker  4c1cb0e95b99f72d55c068ba0255c54d   
3           highway  02e821a3e53cb320790950aab4489e85   
4             cable  e8a8b3a2061aa0e6d7c6b522e9612824   
...             ...                               ...   
7888            cat  1fe90a4aee405e1aa2279442d28803ae   
7889        landing  01794dde3ca2991615f1aa2f63fb22e3   
7890          grape  21ce6f7c5c3d1ad8cf234988c1ad471f   
7891          climb  6c84e79d0595efd99596faa07c4961d0   
7892           hose  88f1fe6cfbcb1a25f25454341c789463   

                                               question            choice_A  \
0     The sanctions against the school were a punish...              ignore   
1     Sammy wanted to go to where the people were. W...          race track   
2     To locate a choker not located in a jewelry bo...       jewelry store   
3     Google Maps and other highway and street GPS s...       united states   
4     What home entertainment equipment requires cable?         radio shack   
...                                                 ...                 ...   
7888                     What are cats often known for?            whiskers   
7889  As he looked out the window, he knew the landi...  apartment building   
7890  If you really wanted a grape, where would you ...              winery   
7891  What would you do to a rock when climb up a cl...                grab   
7892  His compressor needed a new hose, where did he...         garden shed   

             choice_B       choice_C     choice_D     choice_E  label  \
0             enforce  authoritarian      yell at        avoid      0   
1     populated areas     the desert    apartment    roadblock      1   
2                neck    jewlery box  jewelry box     boutique      0   
3              mexico    countryside        atlas       oceans      3   
4          substation        cabinet   television         desk      3   
...               ...            ...          ...          ...    ...   
7888      sharp teeth           purr    four legs  sharp claws      4   
7889        disembark      stairwell      deplane      airport      4   
7890      fruit stand          field      kitchen         food      1   
7891        look down          throw      falling     may fall      0   
7892   hardware store        brothel       garage   greenhouse      1   

               answer  
0              ignore  
1     populated areas  
2       jewelry store  
3               atlas  
4          television  
...               ...  
7888      sharp claws  
7889          airport  
7890      fruit stand  
7891             grab  
7892   hardware store  

[7893 rows x 10 columns]

In [35]:
X_csqa = [i for i in state['train_all_cands']]
X_csqa += [i for i in state['test_all_cands']]
X_csqa += [i for i in state['vall_all_cands']]

In [36]:
Y_csqa = train_data['answer'].tolist() + test_data['answer'].tolist() + vall_data['answer'].tolist()

In [37]:
def repl_Xdrm(X):
    return X[5],

In [56]:
X_all = (X_csqa + X_dream)[:]
Y_all = (Y_csqa + y_dream)[:]

In [57]:
def prep_sample(x):
    x = x.replace(" ","_")
    if x not in graph_state["concepts_maper"]:
        return -100
    else:
        return graph_state["concepts_maper"][x]
    
Y_all = [prep_sample(x) for x in Y_all]

In [58]:
X_st = [i[0] for i in X_all]

In [59]:
Y_all[0]

2929

In [60]:
from sklearn.metrics import accuracy_score
def validate_maper(maper, X,Y):
    preds = []
    for x in X:
        preds.append(maper[x.replace(' ','_')])
    return accuracy_score(preds,Y)

In [61]:
real_maper = graph_state['concepts_maper']
#real_maper = pickle.load(open('csqa_data/maper.pkl', 'rb'))
#inv_maper = graph_state['concepts_maper_inv']

In [62]:
validate_maper(real_maper, [i[5] for i in X_csqa],Y_csqa)

0.0

In [63]:
graph_state.keys()

dict_keys(['graph', 'concepts_maper', 'concepts_maper_inv', 'relation_maper', 'relation_maper_inv', 'embeds'])

In [64]:
inv_maper[238269]

NameError: name 'inv_maper' is not defined

In [73]:
import random
from copy import deepcopy

def update_maper(maper,X,Y,tsh=52.0):
    c = 0
    i = 0
    maper = deepcopy(maper)
    for x,y in zip(X,Y):
        if i <= 10962:
            if random.random() <= 0.02:
                continue
        x = x.replace(' ','_')
        if x in graph_state['concepts_maper']:
            if random.random() <= tsh:
                maper[x] = y
                c += 1
        i += 1
    print(c)
    return maper

In [74]:
new_maper = update_maper(real_maper,X_st,Y_all,tsh=10)

20938


In [75]:
new_maper['pretty_awful']

238269

In [77]:
with open("maper_v2.pkl",'wb') as f:
    pickle.dump(new_maper,f)

In [78]:
validate_maper(new_maper,X_st,Y_all)

0.7392598894087622

In [14]:
import shutil
shutil.make_archive('csqa_data','zip','csqa_data')

'/notebooks/csqa_data.zip'

In [2]:
!jupyter servers

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: bundlerextension contrib dejavu events execute fileid
kernel kernelspec lab labextension labhub migrate nbclassic nbconvert
nbextension nbextensions_configurator notebook run server serverextension
troubleshoot trust

Jupyter command `jupyter-servers` not found.


In [4]:
!jupyter server list

Currently running servers:
http://npii00imaa:8888/?token=d119155750b4b59570c1402091d6aefc :: /notebooks
